In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

# Imports

In [2]:
import numpy as np
from ICARUS.propulsion.engine import Engine
from ICARUS.database import DB

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


#  Load Plane and Engine

In [3]:
engine_dir = "../Data/Engine/Motor_1/"

engine = Engine()
engine.load_data_from_df(engine_dir)
# engine.plot_engine_map()

# Vehicle

In [4]:
from ICARUS.vehicle.plane import Airplane
from ICARUS.mission.mission_vehicle import MissionVehicle

In [5]:
import jax

# Get the percision of the jax library
print()
print('Jax has been configured to use the following devices: ', jax.devices())
jax.config.update("jax_enable_x64", True)


Jax has been configured to use the following devices:  [CpuDevice(id=0)]


In [6]:
plane: Airplane = DB.get_vehicle('final_design')
# plane.visualize(annotate=True)
# plane.main_wing.symmetries

mission_plane = MissionVehicle(
    plane,
    engine,
    solver= "AVL"
)

# from ICARUS.visualization.airplane.db_polars import plot_airplane_polars
# plot_airplane_polars([plane.name])

Loaded Plane Plane Object: final_design


# Compute Trajectory

In [7]:
from ICARUS.geometry.polynomial import h_polynomial_factory
from ICARUS.mission.trajectory.trajectory import MissionTrajectory
from ICARUS.mission.trajectory.integrate import RK45_scipy_integrator
from ICARUS.geometry.cubic_splines import CubicSpline_factory
import jax.numpy as jnp

In [8]:
operating_floor = 12.5
t0 = 0
x0 =  jnp.array([0., 20.])

# Polynomial Trajectory
coeffs = jnp.array(
    [
        x0[1],
        0.2,
        -1/300
    ]
)
traj_fun_1 = h_polynomial_factory(coeffs)

traj_poly  = MissionTrajectory(
    "Polynomial Trajectory", 
    traj_fun_1, 
    vehicle=mission_plane,
    verbosity= 2,
    operating_floor= 12.4
)

# Spline Trajectory
y = [20, 20 , 20, 20, 20, 20,]
x =  jnp.linspace(0, 2000, len(y)+2)
y = jnp.array([x0[1], *y, operating_floor])
spline_i, title = CubicSpline_factory(x,y)

traj_spl = MissionTrajectory(
    title, 
    spline_i, 
    vehicle=mission_plane,
    verbosity= 2,
    operating_floor= 12.4
)
gamma = jnp.arctan(traj_spl.dy_dx(x0[0]))


# Polynomial Trajectory

In [9]:
import jax.numpy as jnp

from ICARUS.mission.trajectory.integrate import RK4systems
# velocity is in the same direction as the derivative of the polynomial
g1 = jnp.arctan(
    traj_spl.dy_dx(x0[0])
)
v0_mag = 20.
v0 = jnp.array([jnp.cos(g1), jnp.sin(g1)]) * v0_mag

dt = 0.01
tend = 200
#t, xs, vs, states = RK4systems(t0, tend, dt, x0, v0, traj_spl )


# Optimization

In [10]:
import matplotlib.pyplot as plt

fig = plt.figure()
axs = fig.subplots(3, 2, squeeze=False).flatten().tolist()

def objective_goal_splines(y, dt = 0.01, plotting = True):
    x =  jnp.linspace(0, 2000, len(y)+2)
    y = jnp.hstack([x0[1], y, operating_floor])

    spline_i, title = CubicSpline_factory(x,y)

    trajectory = MissionTrajectory(
        title, 
        spline_i, 
        vehicle=mission_plane,
        verbosity= 2,
        operating_floor= 12.4
    )
    gamma = jnp.arctan(trajectory.dy_dx(x0[0]))
    v0 = jnp.array([np.cos(gamma), np.sin(gamma)]) * v0_mag


    t, xs, vs, _  = RK4systems(t0, tend, dt, x0, v0, trajectory)
    if plotting:
        trajectory.plot_history(axs,fig)
    # x = [x[0] for x in xs]
    return - jnp.max(xs[:, 0])

In [ ]:
from scipy.optimize import minimize
 
res_splines = minimize(
    objective_goal_splines,
    x0 = [20, 20 , 20, 20, 20, 20,],
    method='BFGS',
    options={'disp': True, 'maxiter' : 3000},
)

y  = res_splines.x
x = jnp.linspace(0, 2000, len(y)+2)

spline_best, title = CubicSpline_factory(x,y)
trajectory_best =  MissionTrajectory(
        title, 
        spline_best, 
        vehicle=mission_plane,
        verbosity= 2,
        operating_floor= 12.4
    )
gamma = jnp.arctan( trajectory_best.dy_dx(x0[0]) )
v0 = jnp.array([np.cos(gamma), np.sin(gamma)]) * v0_mag

t, xs, vs, states = RK4systems(t0, tend, dt, x0, v0, trajectory_best)